In [ ]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

In [ ]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [1]:
import pandas as pd
from sklearn import tree
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.neighbors import KNeighborsClassifier
import joblib

In [3]:
# Read the csv
df = pd.read_csv("exoplanet_data.csv")

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Show first 5 rows
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [5]:
# Set features and ensure they are the same length
X = df.drop('koi_disposition', axis=1)
y = df['koi_disposition'].values.reshape(-1, 1)
print(X.shape, y.shape)

(6991, 40) (6991, 1)


In [6]:
# Label-encode the dataset with one-hot encoder
label_encoder = OneHotEncoder()
label_encoder.fit(y)
encoded_y = label_encoder.transform(y)

encoded_y[1]

<1x3 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [7]:
# Use tree-based estimator to discard irrelevant features 
clf = ExtraTreesClassifier(n_estimators=50)
clf = clf.fit(X, encoded_y.toarray())
clf.feature_importances_  

model = SelectFromModel(clf, prefit=True)
X_new = model.transform(X)
X_new.shape               

(6991, 7)

In [8]:
# Scale the numeric data
scaler = MinMaxScaler()

X_scaler = scaler.fit_transform(X_new)

In [15]:
# Separate the data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X_new, encoded_y, random_state=42)
X_train.shape

(5243, 7)

In [16]:
# Train the model using decision tree
clf = tree.DecisionTreeClassifier()

clf = clf.fit(X_train, y_train.toarray())
clf.score(X_test, y_test.toarray())

0.8209382151029748

In [18]:
# Train the model using random forest
rf = RandomForestClassifier(n_estimators=200)

rf = rf.fit(X_train, y_train.toarray())
rf.score(X_test, y_test.toarray())

0.8581235697940504

In [19]:
# Use GridSearch to tune model parameters
t = tree.DecisionTreeClassifier()

param_grid = {'criterion': ['gini', 'entropy'],
              'splitter': ['best', 'random'],
              'max_leaf_nodes': [13, 47, 69, 420]}
grid_clf = GridSearchCV(t, param_grid, verbose=3)

grid_clf.fit(X_train, y_train.toarray())

grid_clf.score(X_test, y_test.toarray())

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] criterion=gini, max_leaf_nodes=13, splitter=best ................
[CV]  criterion=gini, max_leaf_nodes=13, splitter=best, score=0.887, total=   0.0s
[CV] criterion=gini, max_leaf_nodes=13, splitter=best ................
[CV]  criterion=gini, max_leaf_nodes=13, splitter=best, score=0.879, total=   0.0s
[CV] criterion=gini, max_leaf_nodes=13, splitter=best ................
[CV]  criterion=gini, max_leaf_nodes=13, splitter=best, score=0.883, total=   0.0s
[CV] criterion=gini, max_leaf_nodes=13, splitter=best ................
[CV]  criterion=gini, max_leaf_nodes=13, splitter=best, score=0.865, total=   0.0s
[CV] criterion=gini, max_leaf_nodes=13, splitter=best ................
[CV]  criterion=gini, max_leaf_nodes=13, splitter=best, score=0.864, total=   0.0s
[CV] criterion=gini, max_leaf_nodes=13, splitter=random ..............
[CV]  criterion=gini, max_leaf_nodes=13, splitter=random, score=0.880, total=   0.0s
[CV] criterio

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV]  criterion=gini, max_leaf_nodes=69, splitter=random, score=0.856, total=   0.0s
[CV] criterion=gini, max_leaf_nodes=420, splitter=best ...............
[CV]  criterion=gini, max_leaf_nodes=420, splitter=best, score=0.844, total=   0.0s
[CV] criterion=gini, max_leaf_nodes=420, splitter=best ...............
[CV]  criterion=gini, max_leaf_nodes=420, splitter=best, score=0.822, total=   0.0s
[CV] criterion=gini, max_leaf_nodes=420, splitter=best ...............
[CV]  criterion=gini, max_leaf_nodes=420, splitter=best, score=0.843, total=   0.0s
[CV] criterion=gini, max_leaf_nodes=420, splitter=best ...............
[CV]  criterion=gini, max_leaf_nodes=420, splitter=best, score=0.844, total=   0.0s
[CV] criterion=gini, max_leaf_nodes=420, splitter=best ...............
[CV]  criterion=gini, max_leaf_nodes=420, splitter=best, score=0.847, total=   0.0s
[CV] criterion=gini, max_leaf_nodes=420, splitter=random .............
[CV]  criterion=gini, max_leaf_nodes=420, splitter=random, score=0.85

[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:    0.6s finished


0.8861556064073226

In [20]:
# Save the model

filename = 'model_1.sav'
joblib.dump(grid_clf, filename)

['model_1.sav']